In [1]:

from textwrap import wrap
import shutil
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
import os, warnings
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import numpy as np
import random

print('TensorFlow Version ' + tf.__version__)

def seed_everything(seed = 0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed_everything()

import warnings
warnings.filterwarnings("ignore")

C:\Users\matus\anaconda3\envs\TF\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\matus\anaconda3\envs\TF\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want

TensorFlow Version 2.10.0


In [2]:
image_size = 224
batch_size = 16
n_classes = 3
EPOCHS = 40

train_path = 'binary-dataset/train'
valid_path = 'binary-dataset/train'
test_path = 'binary-dataset/test'


In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                                samplewise_center = True,
                                                                samplewise_std_normalization = True,
                                                                validation_split=0.2,
                                                                dtype='float16')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                               samplewise_center = True,
                                                                samplewise_std_normalization = True,
                                                                 dtype='float16')
train_gen  = train_datagen.flow_from_directory(
train_path,
target_size=(image_size, image_size),
batch_size = batch_size,
seed = 1,
    color_mode = 'rgb',
    shuffle = True,
    class_mode='categorical',
    subset='training') 

# same directory as training data

valid_gen  = train_datagen.flow_from_directory(
    train_path ,
    target_size=(image_size, image_size),
    batch_size = batch_size,
    seed = 1,
    color_mode = 'rgb',
    shuffle = False,
    class_mode='categorical',
    subset='validation',
    )

test_gen  = test_datagen.flow_from_directory(
    test_path ,
    target_size=(image_size, image_size),
    batch_size = batch_size,
    seed = 1,
    color_mode = 'rgb',
    shuffle = False,
    class_mode='categorical',
    )
class_indices_mapping = train_gen.class_indices

Found 2599 images belonging to 2 classes.
Found 649 images belonging to 2 classes.
Found 813 images belonging to 2 classes.


In [4]:
from sklearn.utils import class_weight

train_labels = []
for i in range(len(train_gen)):
    _, labels = train_gen[i]
    train_labels.extend(np.argmax(labels, axis=1))

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_dict = dict(enumerate(class_weights))

In [7]:
class Patches(L.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images = images,
            sizes = [1, self.patch_size, self.patch_size, 1],
            strides = [1, self.patch_size, self.patch_size, 1],
            rates = [1, 1, 1, 1],
            padding = 'VALID',
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [8]:
from vit_keras import vit

vit_model = vit.vit_b16(
        image_size = image_size,
        activation = 'sigmoid',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 2)

In [9]:
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation = tf.keras.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(64, activation = tf.keras.activations.gelu),
        tf.keras.layers.Dense(32, activation = tf.keras.activations.gelu),
        tf.keras.layers.Dense(1, 'sigmoid')
    ],
    name = 'vision_transformer')

model.summary()

KeyboardInterrupt: 

In [ ]:
warnings.filterwarnings("ignore")

learning_rate = 1e-4

optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.2), 
              metrics = ['accuracy'])

STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size



early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True, verbose = 1)
filepath = "checkpoints/no-smoted-saved-model-{epoch:02d}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

History = model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = valid_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 50,
          callbacks = [early_stopping_callbacks, model_checkpoint_callback],
                    class_weight=class_weights_dict)

model.save("models/binary-model.h5")

In [ ]:
def plot_history(item):
    plt.plot(History.history[item], label=item)
    plt.plot(History.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()


plot_history("loss")
plot_history("accuracy")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

predicted_classes = np.argmax(model.predict(test_gen, steps = test_gen.n // test_gen.batch_size + 1), axis = 1)
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())
confusionmatrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize = (10, 10))
sns.heatmap(confusionmatrix, cmap = 'Blues', annot = True, cbar = True ,fmt='g',xticklabels=class_labels, yticklabels=class_labels)

print(classification_report(true_classes, predicted_classes))